
#----
## Goal

The goal in this project is to implement a Decision Tree Classifier.

**Note** Please note that you are allowed to use only those libraries which we have discussed in the class, i.e. numpy, scipy, pandas. You can use scikit-learn only for data splitting.

## Submission Instructions
You are required to submit the original notebook file on the Classsroom (with .ipynb extension), with complete set of outputs. Students failing to do so will get zero marks. 

*Please read each step carefully and understand it fully before proceeding with code writing*

## Plagiarism
Any form of plagiarism will not be tolerated and result in 0 marks.



### Decision Tree Classifier

Now in this assignment we will be implementing the Decision Classifier for Continuous attributes.

You will be testing your implementations with only one split criteria, namely:
 - Information Gain


In [1]:
%pylab inline
import scipy.stats
from collections import defaultdict  # default dictionary 
from random import randint
import math

Populating the interactive namespace from numpy and matplotlib


In [2]:
class Node:
    def __init__(self,purity,klasslabel='',score=0,split=[],fidx=-1):
        self.lchild=None       
        self.rchild=None
        self.klasslabel=klasslabel        
        self.split=split
        self.score=score
        self.fidx=fidx
        self.purity=purity
        
    def set_childs(self,lchild,rchild):
        # YOUR CODE HERE
        self.lchild=lchild
        self.rchild=rchild
        
    def isleaf(self):
        # YOUR CODE HERE
        if self.lchild==None and self.rchild==None:
            return True
        else:
            return False
        
        raise NotImplementedError()
    def isless_than_eq(self, X):
        # YOUR CODE HERE
        raise NotImplementedError()
    def get_str(self):        
        if self.isleaf():
            return 'C(class={},Purity={})'.format(self.klasslabel,self.purity)
        else:
            return 'I(Fidx={},Score={},Split={})'.format(self.fidx,self.score,self.split)
    

In [3]:
# A placeholder class 
# TODO: You have to implement the following class, remember from the lectures that you will 
# need to build a model for each different class you are trying to identify...

In [4]:
## Your code goes here...
# You might need to define auxliary classes for composition.. ?
class DecisionTree:
    ''' Implements the Decision Tree For Classification... '''
    def __init__(self, purityp, exthreshold,maxdepth=10):        
        self.purity=purityp
        self.exthreshold=exthreshold
        self.maxdepth=maxdepth
        pass
    def train(self, X, Y):
        ''' Train Decision Tree using the given 
            X [m x d] data matrix and Y labels matrix
            
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional examples.
            Y: [m x 1] a label vector.
            
            Returns:
            -----------
            Nothing
            '''
        self.nexamples,self.nfeatures=X.shape
        ## now go and train a model for each class...
        # YOUR CODE HERE
        depth=0   
        self.tree = self.build_tree(X,Y,0)
        
        
    def build_tree(self, X, Y, depth):
        # YOUR CODE HERE
        nexamples, nfeatures=X.shape
      
        klasses=np.unique(Y);
        Y_count = np.unique(Y,return_counts=True)
        purity_ = np.max(Y_count[1])/float(len(Y))
        class_ = Y_count[0][np.argmax(Y_count[1])]
        if(purity_>self.purity or nexamples<self.exthreshold or depth>self.maxdepth):
            return Node(klasslabel=class_,purity=purity_)
        bscore = +np.inf
        bfeature = 0
        lchild = []
        rchild = []
        fidx=-1
        threshold = 0
        for i in range(nfeatures):
            sthreshold,sscore,lid,rid = self.evaluate_numerical_attribute(X[:,i],Y)
            if(sscore<bscore):
                bscore = sscore
                lchild = lid
                rchild = rid
                fidx=i
                threshold=sthreshold
        node = Node(purity=purity_,score=sscore,split=threshold,fidx=fidx)
        node.lchild = self.build_tree(X[lchild],Y[lchild],depth+1)
        node.rchild = self.build_tree(X[rchild],Y[rchild],depth+1)
        
        return node
        
        
    def test(self, X):
        
        ''' Test the trained classifiers on the given set of examples 
        
                   
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional test examples.
           
            Returns:
            -----------
                pclass: the predicted class for each example, i.e. to which it belongs
        '''
        pclasses=[]
        nexamples, nfeatures=X.shape
        pass
    
    def evaluate_numerical_attribute(self,feat, Y):

        feat2=np.unique(feat)
       
        splArr=np.copy(feat2)
        for i in range(len(splArr)-1):
            splArr[i]=(splArr[i]+splArr[i+1])/2.0


        j=0    
        labSum=np.zeros(len(np.unique(Y2)))
        splData=[labSum]
       
        for i in range(len(splArr)):
            while j<len(feat) and feat[j]<=splArr[i]:
                labSum[Y2[j]]+=1
               

                j+=1
            splData=np.append(splData,[labSum],axis=0)


        HD=0
        BestIG=-1
        splInd=0

        #Parent Entropy is HD
        for i in range(len(labSum)):
            if (labSum[i]!=0):
                HD+=(labSum[i]/sum(labSum))*math.log((labSum[i]/sum(labSum)),2)
        HD*=-1
       
        for i in range(len(splData)):
            #Child Entropies
            HDy=0
            HDn=0
            leftvec=labSum-splData[i]
            for j in range(len(splData[i])):
                if splData[i][j]!=0:
                    #child entropies
                    HDy+=(splData[i][j]/sum(splData[i]))*math.log((splData[i][j]/sum(splData[i])),2)

                if leftvec[j]!=0:
                    HDn+=(leftvec[j]/sum(leftvec))*math.log(leftvec[j]/sum(leftvec),2)
            HDy*=-1
            HDn*=-1

            HS=((sum(splData[i])/sum(labSum))*HDy)+((sum(leftvec)/sum(labSum))*HDn)
            IG=HD-HS
            if IG>BestIG:
                BestIG=IG
                splInd=i
            
        split_point = splArr[splInd]
        Max_Gain = BestIG
        left_C = feat[feat<split_point]
        right_C = feat[feat>=split_point]
        Xlidx = np.where([feat<split_point])[1]
        Xridx = np.where([feat>=split_point])[1]       

        return split_point,Max_Gain,Xlidx,Xridx
           
    def predict(self, X):
        predClasses=[]
        
        for idx in range(X.shape[0]):
            
            predClasses.append(self._predict(self.tree,X[idx,:]))
        
        return predClasses 
        pass
    
    def _predict(self,node, X):
        if(node.isleaf()==True):
            return node.klasslabel
        idx = node.fidx
        if(X[idx]<node.split):
            return self._predict(node.lchild,X)
        else:
            return self._predict(node.rchild,X)
   
        raise NotImplementedError()

    def __str__(self):
        
        return self.__print(self.tree)        
        
     
    def find_depth(self):
        return self._find_depth(self.tree)
    
    
    def _find_depth(self,node):
        if not node:
            return
        if node.isleaf():
            return 1
        else:
            return max(self._find_depth(node.lchild),self._find_depth(node.rchild))+1
    def __print(self,node,depth=0):
        
        ret = ""

        # Print right branch
        if node.rchild:
            ret += self.__print(node.rchild,depth+1)

        # Print own value
        
        ret += "\n" + ("    "*depth) + node.get_str()

        # Print left branch
        if node.lchild:
            ret += self.__print(node.lchild,depth+1)
        
        return ret

In [5]:
import pandas as pd



# In the first case here, you are asked to train on all the data using only two features and classes. 

In [6]:
#load the data set
data=pd.read_csv('./iris.data')
data.columns=['SepalLength','SepalWidth','PetalLength','PetalWidth','Class']
print data.describe()

       SepalLength  SepalWidth  PetalLength  PetalWidth
count   149.000000  149.000000   149.000000  149.000000
mean      5.848322    3.051007     3.774497    1.205369
std       0.828594    0.433499     1.759651    0.761292
min       4.300000    2.000000     1.000000    0.100000
25%       5.100000    2.800000     1.600000    0.300000
50%       5.800000    3.000000     4.400000    1.300000
75%       6.400000    3.300000     5.100000    1.800000
max       7.900000    4.400000     6.900000    2.500000


In [7]:
# Get your data in matrix
X=np.asarray(data[['SepalLength','SepalWidth','PetalLength','PetalWidth']].dropna())
Y=np.asarray(data['Class'].dropna())
print " Data Set Dimensions=", X.shape, " True Class labels dimensions", Y.shape   
print X

 Data Set Dimensions= (149L, 4L)  True Class labels dimensions (149L,)
[[4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.1 1.5 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2

In [8]:
# here data is being set so that there are only two classes
Y[Y=='Iris-virginica']='Iris-versicolor'
Y

array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor',
       '

# Main Cell

In [9]:
# Get your data in matrix
X=np.asarray(data[['SepalLength','SepalWidth','PetalLength','PetalWidth']].dropna())
Y=np.asarray(data['Class'].dropna())
print " Data Set Dimensions=", X.shape, " True Class labels dimensions", Y.shape   

Y[Y=='Iris-virginica']='Iris-versicolor'
#Shuffling
for i in range(len(X)):
    
    temp=np.copy(X[i])
    temp2=np.copy(Y[i])
    ind=randint(i,len(X)-1)
    X[i]=X[ind]
    Y[i]=Y[ind]
    X[ind]=temp
    Y[ind]=temp2


feat=[0,1] # because we are only using two features in the beginning
dt=DecisionTree(0.95,5,2)
feat=[0,1]
dt.classes=np.unique(Y)
dt.nclasses=len(np.unique(Y))
len(np.unique(Y))
Y2=np.copy(Y)
Y2[Y2=='Iris-setosa']=0
Y2[Y2=='Iris-versicolor']=1
X2=np.copy(X)

trainX=X2[50:]
testX=X2[:50]
trainY=Y2[50:]
testY=Y2[:50]
dt.train(trainX,trainY)
print dt
pclasses=dt.predict(testX)
print "Predicted Classes: ",pclasses
print "Actual Classes: ",testY
print np.sum(pclasses==testY)
print "Accuracy = ", np.sum(pclasses==testY)/float(testY.shape[0])
#split,mingain,Xlidx,Xridx=dt.evaluate_numerical_attribute(X2[:,0],Y2)

 Data Set Dimensions= (149L, 4L)  True Class labels dimensions (149L,)

        C(class=1,Purity=1.0)
    I(Fidx=2,Score=0.00285595262278,Split=6.0)
            C(class=1,Purity=1.0)
        I(Fidx=3,Score=0.00297136292615,Split=1.95)
            C(class=1,Purity=0.746031746032)
I(Fidx=0,Score=0.00572501799574,Split=5.05)
        C(class=0,Purity=0.75)
    I(Fidx=0,Score=0.121071655239,Split=5.0)
            C(class=0,Purity=0.923076923077)
        I(Fidx=0,Score=0.047709111428,Split=4.35)
            C(class=0,Purity=1.0)
Predicted Classes:  [1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0]
Actual Classes:  [1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 1
 0 0 0 1 1 1 1 1 1 1 1 1 0]
43
Accuracy =  0.86


# Testing Entropy and Information Gain

In [10]:
ind=X2[:,0].argsort()
Y2=Y2[ind]
X2=X2[ind]

feat=X2[:,0]
MxGn=1000
print Y2
while (MxGn>0):
    split,maxgain,Xlidx,Xridx = dt.evaluate_numerical_attribute(feat,Y2)
    print "Split Point",split
    print "Max Gain",maxgain
    print "Xlidx",Xlidx
    
    feat=Xlidx
    Y2=Y2[:len(feat)]
    print Y2
    print feat
    MxGn=maxgain
    print "Xridx",Xridx


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]
Split Point 5.55
Max Gain 0.5250492391129916
Xlidx [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1 0]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57]
Xridx [ 58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75
  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
  94  95  96  97  

# Rough

In [11]:
dt.classes=np.unique(Y)
dt.nclasses=len(np.unique(Y))
for i in range(len(X)):
    
    temp=np.copy(X[i])
    temp2=np.copy(Y[i])
    ind=randint(i,len(X)-1)
    X[i]=X[ind]
    Y[i]=Y[ind]
    X[ind]=temp
    Y[ind]=temp2

X,Y
#split,mingain,Xlidx,Xridx=dt.evaluate_numerical_attribute(X[:,0],Y)

(array([[4.6, 3.2, 1.4, 0.2],
        [5.4, 3.7, 1.5, 0.2],
        [7.2, 3.2, 6. , 1.8],
        [4.4, 3. , 1.3, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [6.1, 3. , 4.9, 1.8],
        [6.3, 3.3, 4.7, 1.6],
        [5.4, 3. , 4.5, 1.5],
        [6. , 2.2, 5. , 1.5],
        [6.7, 3.1, 4.4, 1.4],
        [5.1, 2.5, 3. , 1.1],
        [6.4, 3.2, 5.3, 2.3],
        [4.6, 3.6, 1. , 0.2],
        [6.2, 2.9, 4.3, 1.3],
        [6.4, 2.7, 5.3, 1.9],
        [5.7, 2.5, 5. , 2. ],
        [5. , 2.3, 3.3, 1. ],
        [5.4, 3.4, 1.7, 0.2],
        [6.3, 2.9, 5.6, 1.8],
        [5.9, 3. , 5.1, 1.8],
        [6.5, 3. , 5.2, 2. ],
        [5. , 3. , 1.6, 0.2],
        [6.3, 2.8, 5.1, 1.5],
        [5.8, 4. , 1.2, 0.2],
        [6.7, 3.3, 5.7, 2.1],
        [6.8, 3. , 5.5, 2.1],
        [6.9, 3.1, 5.4, 2.1],
        [6.3, 3.3, 6. , 2.5],
        [4.9, 2.5, 4.5, 1.7],
        [5.8, 2.7, 5.1, 1.9],
        [6.7, 2.5, 5.8, 1.8],
        [6.1, 2.8, 4. , 1.3],
        [6.3, 2.7, 4.9, 1.8],
        [4

In [12]:
from nose.tools import assert_almost_equal, assert_equal

assert_equal(split, 5.45)
assert_almost_equal(mingain, 0.38, places=1)

AssertionError: 0 != 5.45

In [ ]:
print dt

In [ ]:
print dt.find_depth()

In [ ]:
# Split your data into training and test-set...
Xtrain=[],Ytrain=[]
Xtest=[],Ytest=[]
print " Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape   
print " Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytest.shape   


In [ ]:
# Lets train a Decision Tree Classifier on Petal Length and Width
feat=[0,1]
dt=DecisionTree(0.95,5)
dt.train(Xtrain[:,feat],Ytrain)
print dt

In [ ]:
#Lets test it on the set of unseen examples...
feat=[0,1]
pclasses=dt.test(Xtest[:,feat])


In [ ]:
print pclasses

Lets see how good we are doing...
==================

In [ ]:
#Lets see how good we are doing, by finding the accuracy on the test set..
print np.sum(pclasses==Ytest)
print "Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0])

In [ ]:
from nose.tools import assert_greater_equal
feat=[0,1]
dt=DecisionTree(0.95,5)
dt.train(Xtrain[:,feat],Ytrain)
pclasses=dt.test(Xtest[:,feat])
acc = np.sum(pclasses==Ytest)/float(Ytest.shape[0])
assert_greater_equal(acc, 0.90)

# Lets Train on all four features....

In [ ]:
#Lets Train on all four features....

# Lets train a Decision Tree Classifier on Petal Length and Width
feat=[0, 1, 2, 3]
dt=DecisionTree(0.95,5)
dt.train(Xtrain[:,feat],Ytrain)
pclasses=dt.test(Xtest[:,feat])
#Lets see how good we are doing, by finding the accuracy on the test set..
print np.sum(pclasses==Ytest)
print "Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0])

# Lets Train on all four features and for all three classes....

In [ ]:
X=np.asarray(data[['SepalLength','SepalWidth','PetalLength','PetalWidth']].dropna())
Y=np.asarray(data['Class'].dropna())
print " Data Set Dimensions=", X.shape, " True Class labels dimensions", Y.shape

In [ ]:
# Split your data into training and test-set... 
# see the documentation of split_data in tools for further information...
Xtrain=[],Ytrain=[]
Xtest=[],Ytest=[]

print " Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape   
print " Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytest.shape   


In [ ]:
feat=[0, 1,2,3]
dt=DecisionTree(0.95,5)
good=[0,1]
dt.train(Xtrain[:,good],Ytrain)
pclasses=dt.test(Xtest[:,good])
#Lets see how good we are doing, by finding the accuracy on the test set..
print np.sum(pclasses==Ytest)
print "Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0])

In [ ]:
from nose.tools import assert_greater_equal

feat=[0, 1, 2, 3]
dt=DecisionTree(0.95,5)
good=[0,1]
dt.train(Xtrain[:,good],Ytrain)
pclasses=dt.te(Xtest[:,feat])
#Lets see how good we are doing, by finding the accuracy on the test set..
acc = np.sum(pclasses==Ytest)/float(Ytest.shape[0])
assert_greater_equal(acc, 0.90)


What can you conclude ?
====================
Please write your observation....

